# **CSE4022: Natural Language Proccessing J Component (Slot:B2)**
## **Project Title:** Automatic Story Generation
### **Team Members:**
1.   Talatala Rahul Reddy  (19BCE1778)
2.   Pulimi Bhargava Reddy (19BCE1342)

#### In this notebook we have loaded all the previously trained LSTM, Bi-directional LSTM, GRU & Bi-directional GRU models and generated stories with a given seed text and also compared the output

### Importing Libraries & Downloading Dependecies

In [1]:
#%tensorflow_version 2.x  #selects the tensorflow 2.0+ version
import tensorflow as tf
import string
import requests
import nltk
import bs4
import re
import nltk
nltk.download('punkt')
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU, Embedding, Bidirectional, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Scraping the Short Stories Website http://www.classicshorts.com/ to train the above models

In [2]:
#return bs4 object
def getPage(url,headers):
    res = requests.get(url, headers=headers)
    try:
        res.raise_for_status()
    except:
        pass
    pageHtml=bs4.BeautifulSoup(res.text,'html.parser')
    return pageHtml

#check if previous scrape already contained the given listing
def getStoryText(link):
    regex = re.compile(r'[\n\r\t]')
    headers = {'User-Agent': 'Mozilla/5.0'}
    pageHtml=getPage(link,headers)
    elems = pageHtml.find_all("div", class_="StoryPara")
    totalText = ""
    for e in elems:
        totalText += regex.sub(" ", e.text.strip())
    return totalText

# Returns an array of dcitionaries - turnip listings
def getListings():
    no_of_stories = 0
    badTitles = {'tlrm','fiddler','frog','ItalianMaster','luck'}
    #f = open("short_stories.txt", "a", encoding='utf-8')
    url = "http://www.classicshorts.com/bib.html"
    headers = {'User-Agent': 'Mozilla/5.0'}
    storyTexts = []
    rawText = ""
    pageHtml=getPage(url,headers)
    elems = pageHtml.find_all("div", class_="biolisting")
    for e in elems:
        currentUrl = "http://www.classicshorts.com/stories/"+e.attrs['onclick'][11:-2]+".html"
        if e.attrs['onclick'][11:-2] not in badTitles:
            pog = getStoryText(currentUrl)
            storyTexts.append(pog)
            no_of_stories = no_of_stories + 1
            #print("No of Stories:"+str(no_of_stories))
            rawText += pog
        if no_of_stories == 100:
            break
        
    #print("Len of story texts:"+str(len(storyTexts)))
    #print("No of stories:"+str(no_of_stories))
    #print(storyTexts)
    #f.write(rawText)
    #f.close()
    return rawText

In [3]:
#fetching the short stories by scraping
stories = getListings()
stories = stories[81:]
len(stories)

1983654

### Sentence Tokenising the short-stories

In [4]:
#performing sentence tokenisation
from nltk.tokenize import sent_tokenize
sentences = sent_tokenize(stories)
len(sentences)

20516

In [5]:
#displays first 3 sentences
sentences[:3]

['My Dear Sir: I am happy to assure you that your anxiety is without reason.',
 'Flemming will be confined to the sofa for three or four weeks, and will have to be careful at first how he uses his leg.',
 'A fracture of this kind is always a tedious affair.']

### Tokenising the short stories and cleaning the data

In [6]:
#cleaning the dataset
def clean_text(sentences):
    #creating tokens from the document
    tokens = []
    for sentence in sentences:
      tokens.extend(nltk.word_tokenize(sentence))
    punc = str.maketrans('','',string.punctuation)
    #this list will remove punctuations
    tokens = [ x.translate(punc) for x in tokens]
    #this list will remove special characters
    tokens = [ x for x in tokens if x.isalpha()]
    #this list converts all the words to lowercase
    tokens = [ x.lower() for x in tokens]
    return tokens

In [7]:
tokens = clean_text(sentences)

In [8]:
tokens[:10]

['my', 'dear', 'sir', 'i', 'am', 'happy', 'to', 'assure', 'you', 'that']

In [9]:
len(tokens) #no of tokens in the 100 short stories

364182

In [10]:
len(set(tokens)) #no of unique words in the 100 short stories

21957

### Generating sequence of tokens

In [11]:
#creating the sequence of tokens
length = 50 + 1 #first 50 words as input and 1 as the output
lines = []

for i in range(length,len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    lines.append(line)
    if i > 110000: #we're only taking few tokens as input for the model
        break

In [12]:
len(lines)

109951

In [13]:
lines[:4]

['my dear sir i am happy to assure you that your anxiety is without reason flemming will be confined to the sofa for three or four weeks and will have to be careful at first how he uses his leg a fracture of this kind is always a tedious affair fortunately',
 'dear sir i am happy to assure you that your anxiety is without reason flemming will be confined to the sofa for three or four weeks and will have to be careful at first how he uses his leg a fracture of this kind is always a tedious affair fortunately the',
 'sir i am happy to assure you that your anxiety is without reason flemming will be confined to the sofa for three or four weeks and will have to be careful at first how he uses his leg a fracture of this kind is always a tedious affair fortunately the bone',
 'i am happy to assure you that your anxiety is without reason flemming will be confined to the sofa for three or four weeks and will have to be careful at first how he uses his leg a fracture of this kind is always a ted

### Converting tokens to numerical values

In [14]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(lines) 
#performing tokenisation on the seq of words (lines) and these seq of words are embedded as integers basically word embedding
#every unique token(word) is given an integer as ML model only works on numerical values
sequences = tokenizer.texts_to_sequences(lines)

In [15]:
tokenizer.word_index 

{'the': 1,
 'and': 2,
 'of': 3,
 'a': 4,
 'to': 5,
 'he': 6,
 'in': 7,
 'was': 8,
 'i': 9,
 'his': 10,
 'it': 11,
 'that': 12,
 'had': 13,
 'with': 14,
 'you': 15,
 'she': 16,
 'on': 17,
 'at': 18,
 'her': 19,
 'as': 20,
 's': 21,
 'for': 22,
 'him': 23,
 'not': 24,
 'but': 25,
 'is': 26,
 'they': 27,
 'from': 28,
 'were': 29,
 'my': 30,
 'said': 31,
 'there': 32,
 'be': 33,
 'all': 34,
 'by': 35,
 'one': 36,
 'have': 37,
 'me': 38,
 'would': 39,
 'nt': 40,
 'an': 41,
 'no': 42,
 'out': 43,
 'up': 44,
 'this': 45,
 'which': 46,
 'what': 47,
 'when': 48,
 'man': 49,
 'we': 50,
 'so': 51,
 'do': 52,
 'them': 53,
 'then': 54,
 'into': 55,
 'or': 56,
 'their': 57,
 'could': 58,
 'now': 59,
 'if': 60,
 'been': 61,
 'like': 62,
 'are': 63,
 'did': 64,
 'about': 65,
 'down': 66,
 'who': 67,
 'your': 68,
 'some': 69,
 'little': 70,
 'over': 71,
 'only': 72,
 'will': 73,
 'eyes': 74,
 'time': 75,
 'more': 76,
 'old': 77,
 'very': 78,
 'back': 79,
 'made': 80,
 'come': 81,
 'again': 82,
 'know':

In [16]:
sequences = np.array(sequences)

In [17]:
sequences[:2]
#it is a 2d array
#rows represent the line of the short stories
#column represent the words (there are 51 cols)

array([[   30,   309,   846,     9,    97,   567,     5,  1890,    15,
           12,    68,  3705,    26,   132,   524,   702,    73,    33,
        11636,     5,     1,  2266,    22,   188,    56,   300,   969,
            2,    73,    37,     5,    33,  5505,    18,   102,   103,
            6, 11633,    10,   371,     4, 11630,     3,    45,   318,
           26,   177,     4,  5504,  3703,  2805],
       [  309,   846,     9,    97,   567,     5,  1890,    15,    12,
           68,  3705,    26,   132,   524,   702,    73,    33, 11636,
            5,     1,  2266,    22,   188,    56,   300,   969,     2,
           73,    37,     5,    33,  5505,    18,   102,   103,     6,
        11633,    10,   371,     4, 11630,     3,    45,   318,    26,
          177,     4,  5504,  3703,  2805,     1]])

In [18]:
sequences.shape #51 cols are words where we take first 50 cols(words) as x and last col(word) as y

(109951, 51)

### Preparing data to that has to fed into the model

In [19]:
#first 50 tokens(words) are x and last tokens is y
X,y = sequences[:,:-1] , sequences[:,-1]
#sequences[:,-1] selects only the last col
#sequences[:,:-1] selects all cols except the last col

In [20]:
X[:2]

array([[   30,   309,   846,     9,    97,   567,     5,  1890,    15,
           12,    68,  3705,    26,   132,   524,   702,    73,    33,
        11636,     5,     1,  2266,    22,   188,    56,   300,   969,
            2,    73,    37,     5,    33,  5505,    18,   102,   103,
            6, 11633,    10,   371,     4, 11630,     3,    45,   318,
           26,   177,     4,  5504,  3703],
       [  309,   846,     9,    97,   567,     5,  1890,    15,    12,
           68,  3705,    26,   132,   524,   702,    73,    33, 11636,
            5,     1,  2266,    22,   188,    56,   300,   969,     2,
           73,    37,     5,    33,  5505,    18,   102,   103,     6,
        11633,    10,   371,     4, 11630,     3,    45,   318,    26,
          177,     4,  5504,  3703,  2805]])

In [21]:
y[:2]

array([2805,    1])

In [22]:
vocabulary_size = len(tokenizer.word_index) + 1 #1 as it's stating from 1 (that word_index is from 1)
vocabulary_size

11638

In [23]:
y = to_categorical(y,num_classes=vocabulary_size)
# a numpy array/vector which has integers that represent different categories is converted into a numpy array/matrix 
#which has binary values and has columns equal to the number of categories in the data.
#X.shape[1] is 50
seq_length = X.shape[1]

### Creating a function to generate stories from a given seed text

In [24]:
def generate_story(model,tokenizer,text_seq_len,seed_text,n_words):
  text = []
  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    #the model expects the input(sentences) to be of same length hence we use the pad_sequences
    #pre it adds 0 before the sentence
    encoded = pad_sequences([encoded],maxlen=text_seq_len,padding='pre')

    pred = model.predict(encoded) 
    y_pred = np.argmax(pred,axis=1)
    predicted_word = ''
    for word,index in tokenizer.word_index.items():
      if index == y_pred:
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return ' '.join(text)

###Loading the LSTM Model

In [25]:
from tensorflow.keras.models import load_model

In [26]:
lstm = load_model('LSTM.h5')

In [27]:
lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            581900    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 dropout (Dropout)           (None, 50, 100)           0         
                                                                 
 lstm_1 (LSTM)               (None, 120)               106080    
                                                                 
 dense (Dense)               (None, 140)               16940     
                                                                 
 dense_1 (Dense)             (None, 11638)             1640958   
                                                                 
Total params: 2,406,278
Trainable params: 2,406,278
Non-

### Loading the Bi-directional LSTM Model

In [28]:
bi_di_lstm = load_model('Bi_di_LSTM.h5')

In [29]:
bi_di_lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            581900    
                                                                 
 bidirectional (Bidirectiona  (None, 50, 200)          120800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 50, 200)           0         
                                                                 
 lstm_1 (LSTM)               (None, 120)               154080    
                                                                 
 dense (Dense)               (None, 140)               16940     
                                                                 
 dense_1 (Dense)             (None, 11638)             1640958   
                                                        

###Loading the GRU Model

In [30]:
gru = load_model('GRU.h5')

In [31]:
gru.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            581900    
                                                                 
 gru (GRU)                   (None, 50, 100)           45600     
                                                                 
 dropout (Dropout)           (None, 50, 100)           0         
                                                                 
 gru_1 (GRU)                 (None, 120)               79920     
                                                                 
 dense (Dense)               (None, 140)               16940     
                                                                 
 dense_1 (Dense)             (None, 11638)             1640958   
                                                                 
Total params: 2,365,318
Trainable params: 2,365,318
Non-

### Loading the Bi-directional GRU Model

In [32]:
bi_di_gru = load_model('Bi_di_GRU.h5')

In [33]:
bi_di_gru.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            581900    
                                                                 
 bidirectional (Bidirectiona  (None, 50, 200)          91200     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 50, 200)           0         
                                                                 
 gru_1 (GRU)                 (None, 120)               115920    
                                                                 
 dense (Dense)               (None, 140)               16940     
                                                                 
 dense_1 (Dense)             (None, 11638)             1640958   
                                                        

### Generating Stories from LSTM Model

In [34]:
seed_text = "The country was in chaos but"
generate_story(lstm, tokenizer, seq_length, seed_text, 50) #generate next 50 words

'humor and men and seemingly retired at the soiled clothing it ll be a good man the first winter carefully falling floundering grasped his knees and and large mustache compelled him giddy to clothes the band were on the mountain bank of western and refrain and today awaiting gilded and'

In [35]:
seed_text = "I walked out of the store dissatisfied and it"
generate_story(lstm, tokenizer, seq_length, seed_text, 50) #generate next 50 words

'and seemingly made him and from the third restaurateur yesterday he was the first time the first winter were sensible gave it there was the sun the young man was so onin the bluff and so provinces in the kitchen i am at the postoffice at least and i am'

In [36]:
seed_text = "The lady was known as a soothsayer - a psychic of sorts. The rubbing of her crystal ball was finalized as she completed her prophecy. Her hands were wrinkled and dirty, a strange companion to the polish of the ball she held in her hands."
generate_story(lstm, tokenizer, seq_length, seed_text, 50) #generate next 50 words

'and cows his pommel of the harbor s hand in the evening and then the banker said quietly because the banker was so concealed by the apparent and besides it was a strip of lemonpeel on the lists of winning numbers johnny dots at the outcome questions or deuced else'

In [37]:
seed_text = "Little Chandler's thoughts ever since lunch-time had been of his meeting with Gallaher, of Gallaher's invitation, and of the great city London where Gallaher lived. He was called Little Chandler because"
generate_story(lstm, tokenizer, seq_length, seed_text, 100) #generate next 100 words

'he saw the handle of the shoulder of the pit of the smallest utterances a top spread out the rear window from the bolivar in order to find him off the cave and he was doingeverything was death at me i am going to be harbors sir than the habit of a letter i suppose the particulars that is no one of the men resurrected out and sit at the first time i had been a hideous impertinence said to mar at get out you ll disturb him upon the door on the lighter and flipped the door and and'

In [38]:
seed_text = "On the morning of the fifth of September, in uniform, his revolver on the table, the colonel addressed his soliders and"
generate_story(lstm, tokenizer, seq_length, seed_text, 100) #generate next 100 words

'had drawn in the darkness and agitating the camp cowling of the desert the captain stopped with emotion irresolution and despondency i would swear at the huts to the east to go to the circumstance which s is the mouse then it is that the general was collected because the young proposition to free his whip ran stonily from the kitchen placed him in a loss i have a notion of st james and consistent with your first cousin husband he said to the smiter so when i cared to pay to the core without needing the gruesome man s'

In [39]:
seed_text = "On Christmas Eve around 1978, when I was a young boy, after having the traditional pastry and cookies and after having sung around the fire, I, with my sister, my mother and my aunts, and Emma Jackman and her son, got into Emma Jackman's car and then later"
generate_story(lstm, tokenizer, seq_length, seed_text, 100) #generate next 100 words

'to him in brighton i leave the ground the children would have heard vines of pull out in the markers were being weak said whitney then but the baron s ancestor thin greatgrandchildren and the most boy he had enjoyed the fox but more penetrating profounder so to speak to the window the wrists bound the perfumes the bewildering medley of color they were blockaded the village of the waterspiders legs were by the sierras and the ditches and coherent whirling diplomacyhardy bestirred lack of fingers trunks benumbed halfblinded somebody else was so hot and cold and clean exhausted themselves'

### Generating Stories from Bi-directional LSTM Model

In [40]:
seed_text = "The country was in chaos but"
generate_story(bi_di_lstm, tokenizer, seq_length, seed_text, 50) #generate next 50 words

'the dime he was not a hospital he amused him to be blowing in his sitting room on the stock of his riflehis first feeling was a random farewell he sprang to the saddle a sinuous line of figures of black hair and prepared to spring however rapid leave the'

In [41]:
seed_text = "I walked out of the store dissatisfied and it"
generate_story(bi_di_lstm, tokenizer, seq_length, seed_text, 50) #generate next 50 words

'might have generally been a shave we can get my nerve with him at the wing commander and she said tom s just so that you want the ideal men it would be concealed in the house and sat down at the instant at bay with worse but become hanley'

In [42]:
seed_text = "The lady was known as a soothsayer - a psychic of sorts. The rubbing of her crystal ball was finalized as she completed her prophecy. Her hands were wrinkled and dirty, a strange companion to the polish of the ball she held in her hands."
generate_story(bi_di_lstm, tokenizer, seq_length, seed_text, 50) #generate next 50 words

'in the soapy water and the experiment and banging of paper with a ring of his ears walked several times writhing and through the horns of the case the man stopped ever got his shoulders his face which led him in recovering in winter he felt sure slippery a diamond'

In [43]:
seed_text = "Little Chandler's thoughts ever since lunch-time had been of his meeting with Gallaher, of Gallaher's invitation, and of the great city London where Gallaher lived. He was called Little Chandler because"
generate_story(bi_di_lstm, tokenizer, seq_length, seed_text, 100) #generate next 100 words

'he could not be partedpeter loved for a winding road lined his footsteps echo and the crackling and banging of fireworks and a young man who vane were sensible of mignonette and a good piece of luck but we rested in the house there for news in little no no provision explained we by midnight of the appearance of the dark i have done the man towards the opinion i learned get down all right i do nt want to be polite words did you get the man to be balanced him even daughter if it can be a transformation'

In [44]:
seed_text = "On the morning of the fifth of September, in uniform, his revolver on the table, the colonel addressed his soliders and"
generate_story(bi_di_lstm, tokenizer, seq_length, seed_text, 100) #generate next 100 words

'apprised him in the gangway of the darkness his only was a terrible cup and the magnetic gaze in the darkness so partly to him a berth for a village and shake word with a sweet little man that holding the manager of a shaker and ice and plenty of glasses but the voice of his throat on board at him with a small steady bandaged stump in him and the prisoner walked into the tepid enchanting delicacy of richard bullen sing o muse of chivalrous men the sacred quest the doughty ropes middle i were tom and i must'

In [45]:
seed_text = "On Christmas Eve around 1978, when I was a young boy, after having the traditional pastry and cookies and after having sung around the fire, I, with my sister, my mother and my aunts, and Emma Jackman and her son, got into Emma Jackman's car and then later"
generate_story(bi_di_lstm, tokenizer, seq_length, seed_text, 100) #generate next 100 words

'known that is to be salted it was extraordinary s a sigh and if i had broken to yourself were italian and when i was a musical instrument and books and sat at the table and read aloud and i m joking he is capable of lemons right and girl for me i have sung songs i have hunted stags and ends it is nt a little chap and she glanced up at the present building i ll give you my lodging i had learned a plague into the tableas i refuse to lose my friend i have a presentiment'

### Generating Stories from GRU Model

In [46]:
seed_text = "The country was in chaos but"
generate_story(gru, tokenizer, seq_length, seed_text, 50) #generate next 50 words

'downcast he got down to her eyes and think that do they get to eating in rubbing fish while mother that bound when they bring him round the doorstep with coatcollar turned out and then then it went out on the bed and went out rather arms and saw the'

In [47]:
seed_text = "I walked out of the store dissatisfied and it"
generate_story(gru, tokenizer, seq_length, seed_text, 50) #generate next 50 words

'had suffered away in any mood s little times violent son and drove out a quarter of a deep black holes off the swamp beyond the deep fields of gate the phalanx of women in their front only fire in which they had passed the difficulty of interest and drinking'

In [48]:
seed_text = "The lady was known as a soothsayer - a psychic of sorts. The rubbing of her crystal ball was finalized as she completed her prophecy. Her hands were wrinkled and dirty, a strange companion to the polish of the ball she held in her hands."
generate_story(gru, tokenizer, seq_length, seed_text, 50) #generate next 50 words

'the panther looked into one of silence when the doctor said she had heard his folly and liked his mind without discomfort was a few weeks there and the doctor later crying and switching up his every morning and sat out her dick had sat between his astonished and stumpy'

In [49]:
seed_text = "Little Chandler's thoughts ever since lunch-time had been of his meeting with Gallaher, of Gallaher's invitation, and of the great city London where Gallaher lived. He was called Little Chandler because"
generate_story(gru, tokenizer, seq_length, seed_text, 100) #generate next 100 words

'that was unnecessary to have at all their own people of the graceful enchanting man buildings and as she had made the carcass at the stranger asked as he observed them that it would be effort to accept the own to force to pieces a wife with hot to dear seen to tell what his dear fellow of a sledgehammer that she had told me now because it s rather to queen at this years he had only used to give him a kitten mamma is at any service of your own daughter well just hearing of its service s'

In [50]:
seed_text = "On the morning of the fifth of September, in uniform, his revolver on the table, the colonel addressed his soliders and"
generate_story(gru, tokenizer, seq_length, seed_text, 100) #generate next 100 words

'friends with pearls but making its discouraged gray dogs his suppers the mouth of his mouth as he remembered that he must lay no voice gasping for look about from this he was climbed behind the people out of you do nt be alarmed said the sea gnashing its hands they were standing in the right to each other six minutes before he was sitting there in the air she was dressed in the lightness of a short corridor glass there on the handle of the table there was no longer the fact and warm that there were no time'

In [51]:
seed_text = "On Christmas Eve around 1978, when I was a young boy, after having the traditional pastry and cookies and after having sung around the fire, I, with my sister, my mother and my aunts, and Emma Jackman and her son, got into Emma Jackman's car and then later"
generate_story(gru, tokenizer, seq_length, seed_text, 100) #generate next 100 words

'as my spirits are not well we need nt jar the old gentleman you much all the first look in the evening s hand the general nodded to him a lot of wild taking a look is there but no more charlie to think it is it she has a job in his legs but he needs put his shoulders and was on the least sign of pain and now struck out tea in the office uncle who grew about when she fell they are all to do with many money filled again do nt understand you pay him the'

### Generating Stories from Bi-directional GRU Model

In [52]:
seed_text = "The country was in chaos but"
generate_story(bi_di_gru, tokenizer, seq_length, seed_text, 50) #generate next 50 words

'they had helped they were back for a mask when he remembered was the child to it had a small patch of congealed blood and the dead s company was letting him midnight the piano on a federal photographer and were out of sorts and social kingdoms your books have'

In [53]:
seed_text = "I walked out of the store dissatisfied and it"
generate_story(bi_di_gru, tokenizer, seq_length, seed_text, 50) #generate next 50 words

'had been devoured grave he cried that he had lost within the character which was three years he could hear johnny called up and with tears and then he dozed offone day when it was like a midshipman and walked in the island of the fort and a piece of'

In [54]:
seed_text = "The lady was known as a soothsayer - a psychic of sorts. The rubbing of her crystal ball was finalized as she completed her prophecy. Her hands were wrinkled and dirty, a strange companion to the polish of the ball she held in her hands."
generate_story(bi_di_gru, tokenizer, seq_length, seed_text, 50) #generate next 50 words

'on every year and that afternoon or until no doubt leapt from my own determination for off de american boy held his champagne he must cook it hundreds of times but i did not know anything he announced gleefully then remembering the family insisted that he and admiral remain behindit'

In [55]:
seed_text = "Little Chandler's thoughts ever since lunch-time had been of his meeting with Gallaher, of Gallaher's invitation, and of the great city London where Gallaher lived. He was called Little Chandler because"
generate_story(bi_di_gru, tokenizer, seq_length, seed_text, 100) #generate next 100 words

'of course we were s means in melancholy in deep and it turned the cliff and his hair was still more as though it were the elder by a matter of minutes and then they were conscious of a small family but no nightlight to make a blessed breach no i ll get up he said somewhat all right we re feeling whom he wondered if he had ever been remembered of course i spoke in a muffled whisper and no doubt flemming these stories oh henry daw was free to see he said he thought malignantly i thought about'

In [56]:
seed_text = "On the morning of the fifth of September, in uniform, his revolver on the table, the colonel addressed his soliders and"
generate_story(bi_di_gru, tokenizer, seq_length, seed_text, 100) #generate next 100 words

'stand quite still more than a baron she proffered no explanation his silence was irritating and made her own plans and myself are haunting me for now that all the first time about whose ferocity was rendered gentle which it was late to be he did not go home in the drama that one could derive true enjoyment and become cultivated and humane but do you suppose the public understands that she used to say what they want to know what i was asked he remarked somewhat later ef i was asked expectations i thought he d throw over some'

In [57]:
seed_text = "On Christmas Eve around 1978, when I was a young boy, after having the traditional pastry and cookies and after having sung around the fire, I, with my sister, my mother and my aunts, and Emma Jackman and her son, got into Emma Jackman's car and then later"
generate_story(bi_di_gru, tokenizer, seq_length, seed_text, 100) #generate next 100 words

'ivan dmitritch thought was little few showing me proffered a poor window the things of silence the band of a small smug town coming to him as if what also could be set advantage of the middle of the big manufacturers of the while and you know but there briefly no longer clearly reply when the commander s men out of human human mother are impossible for me came and s pause i know the incident of the sturgeon marfa went up to the mogilev wilderness he s only a confirmed place of his regiment and i learned this and'